In [13]:
import numpy as np
import pandas as pd
from multiprocessing import Pool
import os

from cloudpoints import load_points
from datasets import CloudDataset, label_names, df_results
from pathlib import Path

In [14]:
data_path = Path("3d_data/")

paris, paris_labels = load_points(data_path / "MiniParis1.ply")
lille1, lille1_labels = load_points(data_path / "MiniLille1.ply")
lille2, lille2_labels = load_points(data_path / "MiniLille2.ply")
dijon = load_points(data_path / "MiniDijon9.ply")
paris_wh = paris[:, 1] <= 20
paris1, paris1_label = paris[paris_wh], paris_labels[paris_wh]
paris2, paris2_label = paris[~paris_wh], paris_labels[~paris_wh]

datasets = [
    CloudDataset("paris1", paris1, paris1_label),
    CloudDataset("paris2", paris2, paris2_label),
    CloudDataset("lille1", lille1, lille1_labels),
    CloudDataset("lille2", lille2, lille2_labels),
    CloudDataset("dijon", dijon)
]
datasets

[CloudDataset<paris1, 1896865 points, labeled>,
 CloudDataset<paris2, 2262453 points, labeled>,
 CloudDataset<lille1, 1901853 points, labeled>,
 CloudDataset<lille2, 2500428 points, labeled>,
 CloudDataset<dijon, 3079187 points, unlabeled>]

In [15]:
inputs = [
    "paris1",
    "paris2",
    "lille1",
    "lille2"
]

def run_GRM_executable(exec_name, input, args=""):
    GRM_path = "GRM_data"
    os.system(f"./{exec_name} {args} < {GRM_path}/{input}_GRM.in > {GRM_path}/{input}_GRM.out")

def run_GRM_executable_all(exec_name, args=""):
    with Pool(len(inputs)) as p:
        p.starmap(run_GRM_executable, [(exec_name, input, args) for input in inputs])
        
def get_clf_labels(input):
    with open(f"GRM_data/{input}_GRM.in") as f:
        lines = [l.strip().split(" ") for l in f.readlines()]
    
    n, _ = lines[0]
    n = int(n)
    P = np.array([[float(f) for f in l] for l in lines[1:n+1]])
    return np.argmax(P, axis=1) + 1

Run your GRM executable : 

In [22]:
%%time
run_GRM_executable_all("repeated_tree_BP.out", args="10.0 0.05 10")

CPU times: user 123 ms, sys: 224 ms, total: 347 ms
Wall time: 2min 15s


In [35]:
true_labels.size / 58546

38.64402350288662

In [34]:
i = 1

GRM_labels = np.array(pd.read_csv(f"GRM_data/{inputs[i]}_GRM.out", header=None)[0])
true_labels = np.array(pd.read_csv(f"GRM_data/{inputs[i]}_labels.txt", header=None)[0])

dataset = datasets[i]

dataset.write_ply_with_labels(f"viz_data/{inputs[i]}_GRM.ply", GRM_labels.astype(np.int32))
dataset.write_ply_with_labels(f"viz_data/{inputs[i]}_true.ply", true_labels.astype(np.int32))

clf_labels = get_clf_labels(inputs[i])
dataset.write_ply_with_labels(f"viz_data/{inputs[i]}_clf.ply", clf_labels.astype(np.int32))

In [33]:
df = df_results(true_labels, GRM_labels)
print("Score:", df["IoU"].mean())
df

Score: 0.6130165278541001


,precision,recall,IoU
1 - Ground,0.972346,0.975110,0.954209
2 - Building,0.681744,0.919892,0.831416
3 - Poles,0.318220,0.419161,0.228451
4 - Pedestrians,0.523903,0.606700,0.431543
5 - Cars,0.538792,0.341440,0.327014
6 - Vegetation,0.917571,0.956145,0.905466


In [27]:
df = df_results(true_labels, clf_labels)
print("Score:", df["IoU"].mean())
df

IndexError: boolean index did not match indexed array along dimension 0; dimension is 1901853 but corresponding boolean dimension is 2262453